In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._


spark: SparkSession = org.apache.spark.sql.SparkSession@2e85c548

In [2]:
import spark.implicits._

case class Transaction(id: Int, country: String, state: String, amount: Int, trans_date: java.sql.Date)

val transactionData: Seq[(Int, String, String, Int, String)] = Seq(
  (121, "US", "approved", 1000, "2018-12-18"),
  (122, "US", "declined", 2000, "2018-12-19"),
  (123, "US", "approved", 2000, "2019-01-01"),
  (124, "DE", "approved", 2000, "2019-01-07"),
  (124, null, "approved", 2000, "2019-01-07")
)

val transactionsDS = transactionData
  .toDF("id", "country", "state", "amount", "trans_date")
  .withColumn("trans_date", to_date(col("trans_date")))
  .as[Transaction]


import spark.implicits._


defined class Transaction
transactionData: Seq[(Int, String, String, Int, String)] = List(
  (121, "US", "approved", 1000, "2018-12-18"),
  (122, "US", "declined", 2000, "2018-12-19"),
  (123, "US", "approved", 2000, "2019-01-01"),
  (124, "DE", "approved", 2000, "2019-01-07"),
  (124, null, "approved", 2000, "2019-01-07")
)
transactionsDS: Dataset[Transaction] = [id: int, country: string ... 3 more fields]

Write an SQL query to find for each month and country, the number of transactions and their total amount, the number of approved transactions and their total amount.

Return the result table in any order.

The query result format is in the following example.

| month    | country | trans_count | approved_count | trans_total_amount | approved_total_amount |
|----------|---------|-------------|----------------|--------------------|-----------------------|
| 2018-12  | US      | 2           | 1              | 3000               | 1000                  |
| 2019-01  | US      | 1           | 1              | 2000               | 2000                  |
| 2019-01  | DE      | 1           | 1              | 2000               | 2000                  |

In [3]:
transactionsDS.show()

+---+-------+--------+------+----------+
| id|country|   state|amount|trans_date|
+---+-------+--------+------+----------+
|121|     US|approved|  1000|2018-12-18|
|122|     US|declined|  2000|2018-12-19|
|123|     US|approved|  2000|2019-01-01|
|124|     DE|approved|  2000|2019-01-07|
|124|   null|approved|  2000|2019-01-07|
+---+-------+--------+------+----------+



In [4]:
transactionsDS.withColumn("month", date_format(col("trans_date"), "yyyy-MM"))
    .withColumn("state", when(col("state") === "approved", 1).otherwise(0))
    .groupBy(col("month"), col("country"))
    .agg(
        count(lit(1)).as("trans_count"),
        sum(col("state")).as("approved_count"),
        sum(col("amount")).as("trans_total_amount"),
        sum(col("amount") * col("state")).as("approved_total_amount")
    )
    .show()

+-------+-------+-----------+--------------+------------------+---------------------+
|  month|country|trans_count|approved_count|trans_total_amount|approved_total_amount|
+-------+-------+-----------+--------------+------------------+---------------------+
|2018-12|     US|          2|             1|              3000|                 1000|
|2019-01|     US|          1|             1|              2000|                 2000|
|2019-01|     DE|          1|             1|              2000|                 2000|
|2019-01|   null|          1|             1|              2000|                 2000|
+-------+-------+-----------+--------------+------------------+---------------------+



In [5]:
spark.stop